PyGaze Analysis

In [13]:
# We load the original results from the file data/4/processed-data/fixationsAll.csv

import pandas as pd

# We load the original results from the file data/4/processed-data/fixationsAll.csv
participants_metadata = pd.read_csv('data/emip_dataset/emip_metadata.csv')

# We load the PyGazeAnalyzer results from the file results/pygaze_fixations_4.csv
pygaze_fixations = pd.read_csv('results/pygaze_fixations_emip_dataset.csv')


# We combine the two dataframes into one by combining the columns where the values of Participant and Task are the same
results_pygaze = pd.merge(participants_metadata, pygaze_fixations, how='inner', left_on=['id'], right_on=['Participant'])
results_pygaze


,id,age,gender,mother_tongue,english_level,visual_aid,makeup,experiment_language,expertise_experiment_language,time_experiment_language,...,stimulus_vehicle,mother_tongue_original,time_experiment_language_original,time_programming_original,other_languages_original,Participant,Task,Fixation Count,Total Fixation Duration [ms],Average Fixation Duration [ms]
0,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,vehicle_java.jpg,247,33581.623,135.957988
1,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_vehicle.jpg,114,17408.333,152.704675
2,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,rectangle_java.jpg,67,6502.875,97.057836
3,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,1,mupliple_choice_rectangle.jpg,128,20457.473,159.824008
4,2,24,male,Slovak,medium,glasses,no,Java,medium,2.0,...,vehicle_java2.jpg,Slovak,2,5,"c++(low), php(medium), delphi(low)",2,rectangle_java2.jpg,131,12446.719,95.013122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,215,22,male,Slovak,medium,no,no,Java,medium,1.0,...,vehicle_java2.jpg,slovak,1,2,C (medium) c# (medium) Java (medium),215,mupliple_choice_rectangle.jpg,113,18223.012,161.265593
860,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,vehicle_java2.jpg,296,29761.160,100.544459
861,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,mupliple_choice_vehicle.jpg,72,8699.441,120.825569
862,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,216,rectangle_java2.jpg,150,13818.919,92.126127


In [6]:
from scipy.stats import wilcoxon
from scipy.stats import mannwhitneyu

def show_results1(results):
	# We calculate the mean and the standard deviation of the fixation count for each CategoryName
		# We first calculate the mean for each participant
	results_mean_per_subject = results.groupby(['CategoryName', 'Subject']).mean(numeric_only=True)[["Fixation Count"]]
	results_std_per_subject = results.groupby(['CategoryName', 'Subject']).std(numeric_only=True)[["Fixation Count"]]

	print(results_mean_per_subject)

	results_mean = results_mean_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count"]]
	results_std = results_std_per_subject.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count"]]

	print(results_mean)


	#results_mean = results.groupby(['CategoryName']).mean(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]
	#results_std = results.groupby(['CategoryName']).std(numeric_only=True)[["Fixation Count", "Total Fixation Duration [ms]"]]

	# Apply Pairwise Comparisons Using Non-Parametric Wilcox Tests (α = 0.05)
	# TODO: check if there is a way to use wilcox 

	stat_list_vs_mental_fix_count, p_list_vs_mental_fix_count = wilcoxon(results_mean_per_subject['Fixation Count'], results_std_per_subject['Fixation Count'])

	# Use mannwhitneyu for the other comparison
	stat_tree_vs_mental_fix_count, p_tree_vs_mental_fix_count = mannwhitneyu(results[results['CategoryName'] == "Tree"]['Fixation Count'], results[results['CategoryName'] == "Mental"]['Fixation Count'])

	stat_list_vs_mental_fix_time, p_list_vs_mental_fix_time = mannwhitneyu(results[results['CategoryName'] == "List"]['Total Fixation Duration [ms]'], results[results['CategoryName'] == "Mental"]['Total Fixation Duration [ms]'])

	# Use mannwhitneyu for the other comparison
	stat_tree_vs_mental_fix_time, p_tree_vs_mental_fix_time = mannwhitneyu(results[results['CategoryName'] == "Tree"]['Total Fixation Duration [ms]'], results[results['CategoryName'] == "Mental"]['Total Fixation Duration [ms]'])

	# We create a table to show the results

	# We divide the fixation time by 1000 to get the fixation time in seconds
	results_mean['Total Fixation Duration [ms]'] = results_mean['Total Fixation Duration [ms]']/1000
	results_std['Total Fixation Duration [ms]'] = results_std['Total Fixation Duration [ms]']/1000

	data = [

		[
			'Fixation Time (s)', 
			f"{'%.2f' % results_mean.loc['List']['Total Fixation Duration [ms]']}({'%.2f' % results_std.loc['List']['Total Fixation Duration [ms]']})",
			f"{'%.2f' % results_mean.loc['Mental']['Total Fixation Duration [ms]']}({'%.2f' % results_std.loc['Mental']['Total Fixation Duration [ms]']})",
			f"{'%.2f' % results_mean.loc['Tree']['Total Fixation Duration [ms]']}({'%.2f' % results_std.loc['Tree']['Total Fixation Duration [ms]']})",
			'%.3f' % p_list_vs_mental_fix_time,
			'%.3f' % p_tree_vs_mental_fix_time
		],
		[
			'Fixation Count', 
			f"{'%.0f' % results_mean.loc['List']['Fixation Count']} ({'%.0f' % results_std.loc['List']['Fixation Count']})", 
			f"{'%.0f' % results_mean.loc['Mental']['Fixation Count']} ({'%.0f' % results_std.loc['Mental']['Fixation Count']})", 
			f"{'%.0f' % results_mean.loc['Tree']['Fixation Count']} ({'%.0f' % results_std.loc['Tree']['Fixation Count']})", 
			'%.3f' % p_list_vs_mental_fix_count, 
			'%.3f' % p_tree_vs_mental_fix_count
		]

		
	]
	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'List', 'Mental', 'Tree', 'List vs Mental p', 'Tree vs Mental p']
	return data_df


In [7]:

from pandas import Interval

def show_results2(results):

	results_std = results[["gender", "age", "Fixation Count", "Average Fixation Duration [ms]"]].std(numeric_only=True)

	# Group the results by the different values of gender
	results_sex = results.groupby("gender")[["Fixation Count", "Average Fixation Duration [ms]"]]
	results_sex_mean = results_sex.mean(numeric_only=True)
	results_sex_std = results_sex.std(numeric_only=True)

	# Group the results by ages between 18-22 and 23-27
	results_age = results.groupby(pd.cut(results["age"], [18, 22, 27]))[["Fixation Count", "Average Fixation Duration [ms]"]]
	results_age_mean = results_age.mean(numeric_only=True)
	results_age_std = results_age.std(numeric_only=True)

	# Perform non-parametric Wilcoxon tests (α =0.05) 
	stat_gender_fix_count, p_gender_fix_count = mannwhitneyu(results[results['gender'] == "male"]['Fixation Count'], results[results['gender'] == "female"]['Fixation Count'])
	stat_gender_fix_time, p_gender_fix_time = mannwhitneyu(results[results['gender'] == "male"]['Average Fixation Duration [ms]'], results[results['gender'] == "female"]['Average Fixation Duration [ms]'])

	stat_age_fix_count, p_age_fix_count = mannwhitneyu(results[results['age'] <= 22]['Fixation Count'], results[results['age'] > 22]['Fixation Count'])
	stat_age_fix_time, p_age_fix_time = mannwhitneyu(results[results['age'] <= 22]['Average Fixation Duration [ms]'], results[results['age'] > 22]['Average Fixation Duration [ms]'])

	# We create a table to show the results
	data = [
		[
			'Avg. Fix. Duration (ms)', 
			f"{'%.2f' % results_sex_mean.loc['male']['Average Fixation Duration [ms]']}({'%.2f' % results_sex_std.loc['male']['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_sex_mean.loc['female']['Average Fixation Duration [ms]']}({'%.2f' % results_sex_std.loc['female']['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_age_mean.loc[Interval(18, 22)]['Average Fixation Duration [ms]']}({'%.2f' % results_age_std.loc[Interval(18, 22)]['Average Fixation Duration [ms]']})",
			f"{'%.2f' % results_age_mean.loc[Interval(22, 27)]['Average Fixation Duration [ms]']}({'%.2f' % results_age_std.loc[Interval(22, 27)]['Average Fixation Duration [ms]']})",
			'%.3f' % p_gender_fix_time,
			'%.3f' % p_age_fix_time
		],
		[
			'Fixation Count', 
			f"{'%.0f' % results_sex_mean.loc['male']['Fixation Count']} ({'%.0f' % results_sex_std.loc['male']['Fixation Count']})", 
			f"{'%.0f' % results_sex_mean.loc['female']['Fixation Count']} ({'%.0f' % results_sex_std.loc['female']['Fixation Count']})", 
			f"{'%.0f' % results_age_mean.loc[Interval(18, 22)]['Fixation Count']} ({'%.0f' % results_age_std.loc[Interval(18, 22)]['Fixation Count']})", 
			f"{'%.0f' % results_age_mean.loc[Interval(22, 27)]['Fixation Count']} ({'%.0f' % results_age_std.loc[Interval(22, 27)]['Fixation Count']})", 
			'%.3f' % p_gender_fix_count, 
			'%.3f' % p_age_fix_count
		]
	]

	data_df = pd.DataFrame(data)
	data_df.columns = ['', 'Men', 'Women', '18-22', '23-27', 'Gender p', 'Age p']
	return data_df



PyGaze vs Ogama

In [8]:
# Function to make the file compatible with Ogama
import pandas as pd

# We load the original data from the file data/4/formatted-raw-data/151/ogama.txt
data = pd.read_csv('data/4/formatted-raw-data/153/ogama.txt', sep=',', skiprows=0, encoding='utf-16')

# Change the values of the ImageName column to remove the last 4 characters
data[' ImageName'] = data[' ImageName'].str[:-4]

# We remove all rows where Included? is "N"
data = data[data[' Included?'] == "Y"]

# Save it again as ogama2.csv
data.to_csv('data/4/formatted-raw-data/153/changed.csv', index=False)

Import data automatically into Ogama by manipulating the database 

In [4]:
import pandas as pd
import os
import sqlite3
import numpy as np
# Need to register numpy types to be able to save them in the database, as SQL only supports int8
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

# Path to database
database_path = 'G:\\Dokumente\\OgamaExperiments\\ExperimentEMIP\\Database\\ExperimentEMIP.db'

trial_to_num = {
		'mupliple_choice_rectangle.jpg': '1',
		'mupliple_choice_vehicle.jpg': '2',
		'rectangle_java.jpg': '3',
		'rectangle_java2.jpg': '4',
		'rectangle_python.jpg': '5',
		'rectangle_scala.jpg': '6',
		'vehicle_java.jpg': '7',
		'vehicle_java2.jpg': '8',
		'vehicle_python.jpg': '9',
		'vehicle_scala.jpg': '10',
	}

def add_to_sql(c, df_msg, subject, last_task_name):

	#df_msg = df_msg[(df_msg['L Validity'] == 1) & (df_msg['R Validity'] == 1)].copy()
	df_msg = df_msg[(df_msg['R Validity'] == 1)].copy()

	if df_msg.size == 0:
		print("df_msg is empty", last_task_name)
		return

	# Define parameters for fixation detection
	x_left = df_msg.loc[:,('L Raw X [px]')].astype(float)
	#x_right = df_msg.loc[:,('R Raw X [px]')]
	x_right = df_msg.loc[:,('R POR X [px]')]
	#df_msg.loc[:,('X')] = (x_left + x_right) / 2
	df_msg.loc[:,('X')] = x_right

	y_left = df_msg['L Raw Y [px]']
	#y_right = df_msg['R Raw Y [px]']
	y_right = df_msg.loc[:,('R POR Y [px]')]
	#df_msg.loc[:,('Y')] = (y_left + y_right) / 2
	df_msg.loc[:,('Y')] = y_right
	

	# Add a table for the subject called Subject + 'RawData'
	c.execute("CREATE TABLE IF NOT EXISTS ["+ subject +"Rawdata] ([ID] integer PRIMARY KEY AUTOINCREMENT NOT NULL,[SubjectName] varchar(50) NOT NULL COLLATE NOCASE, [TrialSequence] integer NOT NULL, [Time] integer NOT NULL, [PupilDiaX] float, [PupilDiaY] float, [GazePosX] float, [GazePosY] float, [MousePosX] float, [MousePosY] float, [EventID] integer)")

	# Add data in the following scheme:
	# SubjectName = subject
	# TrialSequence = ' ImageName'
	# Time = ' StartTime'
	# GazePosX = ' X'
	# GazePosY = ' Y'

	# For each row in the dataframe
	for index, row in df_msg.iterrows():
		# Insert the data into the table
		c.execute("INSERT INTO ["+ subject +"Rawdata] (SubjectName, TrialSequence, Time, GazePosX, GazePosY) VALUES (?, ?, ?, ?, ?)", (subject, trial_to_num[last_task_name], row['Time'], row['X'], row['Y']))

	trialstarttime = df_msg['Time'].min()
	# Endtime - Starttime + 4 ms for the duration of the last measuremnt
	trialduration = df_msg['Time'].max() - df_msg['Time'].min() + 4
	if np.isnan(trialduration):
		print("trialduration is nan", trialstarttime, df_msg['Time'].max(), df_msg['Time'].min())
		print(df_msg)

	trialduration = int(trialduration)
	c.execute("INSERT INTO Trials (SubjectName, TrialID, TrialName, TrialSequence, Category,  TrialStartTime, Duration) VALUES (?, ?, ?, ?, ?, ?, ?)", (subject, trial_to_num[last_task_name], last_task_name, trial_to_num[last_task_name], "",trialstarttime, trialduration))

def import_data_into_ogama(path, subject):

	print("prepare_tobii_data", path)

	# Load the Tobii eye tracker data into a Pandas DataFrame and skip lines that start with ## as they are comments

	df = None
	possible_skipped_rows = [37, 32, 41, 45]
	counter = 0
	while df is None:
		try:
			df = pd.read_csv(path, delimiter='\t', low_memory=False, on_bad_lines='skip', skiprows=possible_skipped_rows[counter])
			df = df[['Time', 'Type', 'L Raw X [px]', 'L Raw Y [px]', 'R Raw X [px]', 'R Raw Y [px]', 'L Validity', 'R Validity', 'R POR X [px]', 'R POR Y [px]']]
		except KeyError as e:
			counter = counter + 1
			df = None
			if counter >= len(possible_skipped_rows):
				print("no possible skipped rows worked", e)
				break
		else:
			#print("possible skipped rows worked", possible_skipped_rows[counter])
			break

	df['Type'] = df['Type'].astype(str)
	df = df.fillna(0.0)

	# Normalize time
	df['Time'] = df['Time'] - df['Time'].min()
	# Time conversion from microseconds to milliseconds
	df['Time'] = df['Time'] / 1000
	# Time as integer
	df['Time'] = df['Time'].astype('int32')

	# Get the row numbers where Type is 'MSG'
	msg_rows = df[df['Type'] == 'MSG'].index

	# For each msg_row, split the df into two dataframes, one before the msg_row and one after
	# Run the analysis for each of those split dataframes as they represent different tasks
	df_after_msg = df
	last_task_name = None
	last_task_row_number = 0
	removed_task_names = {
		'instruction_calibration.jpg': True,
		'instruction_comprehension.jpg': True,
	}


	# Connect to database
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	for msg_row in msg_rows:
		current_task_name = df['L Raw X [px]'][msg_row].split('Message: ')[1]
		if '.jpg' in current_task_name:			
			if last_task_name is not None and last_task_name not in removed_task_names:
				
				df_msg = df[last_task_row_number:msg_row]
				if df_msg.size > 0:
					add_to_sql(c, df_msg, subject, last_task_name)
				else:
					print("df_msg is empty", last_task_name, last_task_row_number, msg_row)
				

			last_task_name = current_task_name
			last_task_row_number = msg_row

	df_msg = df[last_task_row_number:]
	add_to_sql(c, df_msg, subject, last_task_name)
	

	# Add subject to the Subjects table
	c.execute("INSERT INTO Subjects (SubjectName) VALUES (?)", (subject,))

	# Save the database
	conn.commit()

def drop_all_subject_tables():
	# Connect to database
	import sqlite3
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	for i in range(1, 217):
		# Drop the table named S + str(i) + "RawData"
		c.execute("DROP TABLE IF EXISTS S" + str(i) + "Rawdata")

	# Delete entries from Trials
	c.execute("DELETE FROM Trials")

	# Delete entries from Subjects
	c.execute("DELETE FROM Subjects")

	# Save the database
	conn.commit()

drop_all_subject_tables()

#for i in range(22, 23):
for i in range(1, 217):
	# If the file i+'_rawdata.tsv' exists
	filepath = 'data/emip_dataset/rawdata/'+ str(i) + '_rawdata.tsv'
	if os.path.isfile(filepath):
		# Import the data into the database
		print("import_data_into_ogama", filepath, 'S' + str(i))
		import_data_into_ogama(filepath, 'S' + str(i))

import_data_into_ogama data/emip_dataset/rawdata/1_rawdata.tsv S1
prepare_tobii_data data/emip_dataset/rawdata/1_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/2_rawdata.tsv S2
prepare_tobii_data data/emip_dataset/rawdata/2_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/3_rawdata.tsv S3
prepare_tobii_data data/emip_dataset/rawdata/3_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/4_rawdata.tsv S4
prepare_tobii_data data/emip_dataset/rawdata/4_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/5_rawdata.tsv S5
prepare_tobii_data data/emip_dataset/rawdata/5_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/6_rawdata.tsv S6
prepare_tobii_data data/emip_dataset/rawdata/6_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/7_rawdata.tsv S7
prepare_tobii_data data/emip_dataset/rawdata/7_rawdata.tsv
import_data_into_ogama data/emip_dataset/rawdata/8_rawdata.tsv S8
prepare_tobii_data data/emip_dataset/rawdata/8_rawdata.tsv


Now enter Ogama, load the experiment that is using this database and run the fixation algorithm.

Import the fixations from the database and calculate our results

In [11]:
import pandas as pd
import os

# Path to database
database_path = 'G:\\Dokumente\\OgamaExperiments\\ExperimentEMIP\\Database\\ExperimentEMIP.db'

total_results = pd.DataFrame()

def calculate_results_for_subject(subject):
	# Connect to database
	import sqlite3
	conn = sqlite3.connect(database_path)
	c = conn.cursor()

	# Export all fixations for the subject
	c.execute("SELECT * FROM GazeFixations WHERE SubjectName = ?", (subject,))
	fixations = c.fetchall()


	if len(fixations) == 0:
		# Create an empty dataframe with the columns ID, SubjectName, TrialID, TrialSequence, CountInTrial, StartTime, Length, PosX, PosY
		fixations_df = pd.DataFrame(columns=['ID', 'SubjectName', 'TrialID', 'TrialSequence', 'CountInTrial', 'StartTime', 'Length', 'PosX', 'PosY'])
	else:
		# Save it into a dataframe
		fixations_df = pd.DataFrame(fixations)

		# Set the column names of the dataframe
		fixations_df.columns = ['ID', 'SubjectName', 'TrialID', 'TrialSequence', 'CountInTrial', 'StartTime', 'Length', 'PosX', 'PosY']

	# For each TrialID, we calculate the total fixation count, fixation duration and average fixation duration
	results = pd.DataFrame()
	results['Total Fixation Count'] = fixations_df.groupby('TrialID')['ID'].count()
	results['Total Fixation Duration'] = fixations_df.groupby('TrialID')['Length'].sum()
	results['Average Fixation Duration'] = fixations_df.groupby('TrialID')['Length'].mean()
	results['Subject'] = subject


	return results

	


for i in range(1, 217):
	# If the folder i exists
	path = 'data/emip_dataset/rawdata/' + str(i) + '_rawdata.tsv'
	if os.path.isfile(path):
		# Append to the total_results dataframe
		total_results = pd.concat([total_results, calculate_results_for_subject('S' + str(i))])


total_results['Subject'] = total_results['Subject'].str[1:].astype(int)

# We load the original results from the file data/4/processed-data/fixationsAll.csv
participants_metadata = pd.read_csv('data/emip_dataset/emip_metadata.csv')


print("total_results", total_results)

print(total_results.dtypes)
print(participants_metadata.dtypes)

# We combine the two dataframes into one by combining the columns where the values of Participant and Task are the same
results_ogama = pd.merge(participants_metadata, total_results, how='inner', left_on=['id'], right_on=['Subject'])
# Rename the columns Fixation Count, Total Fixation Duration and Average Fixation Duration
results_ogama = results_ogama.rename(columns={'Total Fixation Count': 'Fixation Count', 'Total Fixation Duration': 'Total Fixation Duration [ms]', 'Average Fixation Duration': 'Average Fixation Duration [ms]'})

results_ogama


total_results          Total Fixation Count  Total Fixation Duration  \
TrialID                                                  
1                          94                    25315   
2                          87                    20467   
3                          49                     9865   
7                         194                    43131   
1                         131                    27978   
...                       ...                      ...   
8                         230                    63208   
1                          79                    22713   
2                          75                    13322   
4                         148                    25092   
8                         307                    56132   

         Average Fixation Duration  Subject  
TrialID                                      
1                       269.308511        1  
2                       235.252874        1  
3                       201.326531        1  
7

,id,age,gender,mother_tongue,english_level,visual_aid,makeup,experiment_language,expertise_experiment_language,time_experiment_language,...,order_vehicle,stimulus_vehicle,mother_tongue_original,time_experiment_language_original,time_programming_original,other_languages_original,Fixation Count,Total Fixation Duration [ms],Average Fixation Duration [ms],Subject
0,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,1,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,94,25315,269.308511,1
1,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,1,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,87,20467,235.252874,1
2,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,1,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,49,9865,201.326531,1
3,1,63,male,Czech,high,glasses,no,Java,high,30.0,...,1,vehicle_java.jpg,czech,30y,40+,CoffeeScript (high) JavaScript (high) Python (...,194,43131,222.324742,1
4,2,24,male,Slovak,medium,glasses,no,Java,medium,2.0,...,2,vehicle_java2.jpg,Slovak,2,5,"c++(low), php(medium), delphi(low)",131,27978,213.572519,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,215,22,male,Slovak,medium,no,no,Java,medium,1.0,...,1,vehicle_java2.jpg,slovak,1,2,C (medium) c# (medium) Java (medium),230,63208,274.817391,215
854,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,1,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,79,22713,287.506329,216
855,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,1,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,75,13322,177.626667,216
856,216,22,female,Thai,high,glasses,no,Java,none,0.0,...,1,vehicle_java2.jpg,Thai,6 weeks,6 weeks,Low,148,25092,169.540541,216


In [14]:
data_df = show_results2(results_pygaze)
data_df

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),175.54(44.26),158.38(46.66),175.38(46.10),170.76(44.37),0.000,0.051
1,Fixation Count,180 (134),140 (118),176 (138),174 (117),0.000,0.729


In [12]:
data_df = show_results2(results_ogama)
data_df

,,Men,Women,18-22,23-27,Gender p,Age p
0,Avg. Fix. Duration (ms),287.60(79.18),307.50(283.71),293.73(121.96),282.78(79.07),0.002,0.158
1,Fixation Count,134 (99),114 (95),134 (105),132 (91),0.003,0.769


![Results Study 4 2](images/results_study_4_2.png)